In [13]:
# Load libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api as sm

import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn

listings_df = pd.read_csv("data/yvr_listing_data_cleaned.csv")

In [12]:
### Deal with multicollinearity: Delete variables with highest VIF (From QM Practical 4)

# calculating VIF
# This function is adjusted from: https://stackoverflow.com/a/51329496/4667568
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, thresh=3):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: (default 5) the threshould VIF value. If the VIF of a variable is greater than thresh, it should be removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constatnt item to the data. add_constant is a function from statsmodels (see the import above)
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const
        vif_df = vif_df.drop('const')
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        # If there are multiple variabels with VIF>thresh, only one of them is removed. This is because we want to keep as many variables as possible
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

df_predictors_selected_VIF = drop_column_using_vif_(listings_df, threshhold=3)

In [ ]:
### Create logistic regression model (From Juanes)

"""
Create a multiple logistic regression model to predict legal_listing using the following independent variables:
review_scores_rating, price, instant_bookable
"""
# Convert instant_bookable to a boolean variable
listings_df['instant_bookable'] = listings_df['instant_bookable'].map({'t': 1, 'f': 0})

# Convert price to a float variable
listings_df['price'] = listings_df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# Prepare the data
X = listings_df[['review_scores_rating', 'price', 'instant_bookable']]
y = listings_df['legal_listing']

# Add constant to the independent variables
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Print the summary of the model
print(result.summary())

# Using the results of a logistic regression model, predict the probability of a listing being legal
# given the following values of the independent variables:
# review_scores_rating = 4.5, price = $256, instant_bookable = False
print("\n       Prediction Results")
print("Probability of being legal:", result.predict([1, 4.5, 256, 0])[0])

# Show formula for the model
print("\n       Model Formula")
print("logit(p) = ", result.params[0], "+", result.params[1], "* review_scores_rating +", result.params[2], "* price +", result.params[3], "* instant_bookable")

# Apply formula to calculate the probability of being legal (sigmoid function)
print("\n       Prediction Results")
print("Probability of being legal:", 1 / (1 + 2.71828 ** -(result.params[0] + result.params[1] * 4.5 + result.params[2] * 256 + result.params[3] * 0)))

In [ ]:
### Interpret the logistic regression model

## 1. Model fitness: R-squre, adjusted R-squre, p-value (From Juanes)
"""
Using the logistic regression Wald test, see if there is a connection between review_scores_rating (independent variable)
and legal_listing (dependent variable). The null hypothesis is that there is no connection between the two variables.
review_scores_rating is a continuous variable from 0 to 5, while legal_listing is a categorial boolean variable.
"""
# Remove rows where review_scores_rating is NaN
listings_df = listings_df[listings_df['review_scores_rating'].notna()]

# Prepare the data
X = listings_df['review_scores_rating']
y = listings_df['legal_listing']

# Fit the logistic regression model
model = sm.Logit(y, sm.add_constant(X))
result = model.fit()

print(result.summary())

# Perform the Wald test
wald_test = result.wald_test("review_scores_rating = 0", scalar=True)

# Print the Wald test results
print("\n       Wald Test Results")
print("Test Statistic:", wald_test.statistic)
print("P-value:", wald_test.pvalue)
print("Degrees of Freedom:", wald_test.df_denom)

## or 1. Model fitness: R-squre, adjusted R-squre, p-value (From QM Practical 4)
model_listings_df = sm.OLS(endog=listing_df[['review_scores_rating']], exog=sm.add_constant(df_predictors_selected_VIF)).fit()
model_listings_df.summary()

In [ ]:
### Interpret the logistic regression model (From QM Practical 4)
## 2. Residual analysis: Linear relationship: rainbow test
#                      Independent values: Durbin-Watson test
#                      normally distributed values: Jarque Bera test, 
#                      equal variance: Goldfeld-Quandt homoskedasticity test
# As each of the p-value is less than 0.05, we can reject null hypothesis

# Linear relationship: rainbow test
test_rainbow = statsmodels.stats.diagnostic.linear_rainbow(model_listings_df)
# This function returns a tuple consisting of two values: the test statistic based on the F test and the pvalue of the test
# Note that these two values are not named. Therefore, you need to know the order before accessing these two values.
print("The p value of the rainbow test: {:.4f}".format(test_rainbow[1]))

#Independent values: Durbin-Watson test
test_dw = statsmodels.stats.stattools.durbin_watson(model_listings_df.resid)
print("Durbin-Watson test statistic is: {:.4f}".format(test_dw))

#normally distributed values: Jarque Bera test
test_JB = statsmodels.stats.stattools.jarque_bera(model_listings_df.resid)
print("The p value of the Jarque Bera test: {:.4f}".format(test_JB[1]))

#Equal variance: Goldfeld-Quandt homoskedasticity test
statsmodels.stats.diagnostic.het_goldfeldquandt(model_listings_df.model.endog, model_listings_df.model.exog)

In [ ]:
### Interpret the logistic regression model
## 3. Coefficients
# As in the third cell "Create logistic regression model (From Juanes)".
# Using coefficients to evaluate each variable's weight, direction, statistical significance, Variable Standardization when we trying to interpret the model